In [ ]:
!pip install tweepy==4.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [ ]:
import tweepy
import pandas as pd


# Set Credentials

# Consumer Keys:
consumer_key = "xaOJBR"
consumer_secret = "iZzKobAYmWtUfobIz"

# Authentication Tokens:
bearer_token = "AAabcJG5f1Yz0r"
access_token = "150437np3yx03zSH"
access_token_secret = "y4IPeqHvoBEt4qIDSi4Z"

In [ ]:
# authenticate with the Twitter API, and create the 'api' object which we'll use to access the Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Il est important de noter que l'accès aux informations de nombre de citations est limité pour les tweets qui ont été publiés avant novembre 2020, et certaines informations peuvent être absentes pour certains tweets.

In [ ]:
def search_by_keyword(api, keyword):
    
    # create a pandas dataframe to save the fetched tweets
    df = pd.DataFrame(columns=['id', 'created_at', 'username', 'location', 
                               'followers', 'following', 'totaltweetsbyuser', 
                               'retweetcount', 'like_count', 'impression_count', 
                               'hashtag_list', 'mention_list', 'text'])
    
    # run a query to twitter
    tweets = tweepy.Cursor(api.search_tweets, q=keyword, lang="en",
                           tweet_mode='extended').items(100) 
    # ** note that we are only fetching the first 10 tweets,
    # as to not exceed the limit of the free API tier **
    
    #save the tweets in our dataframe.
    list_tweets = [tweet for tweet in tweets] 
    for tweet in list_tweets: 

        id = tweet.id
        created_at = tweet.created_at
        username = tweet.user.screen_name 
        location = tweet.user.location 

        followers = tweet.user.followers_count
        following = tweet.user.friends_count 
        totaltweetsbyuser = tweet.user.statuses_count 

        retweetcount = tweet.retweet_count 
        like_count = tweet.favorite_count
        impression_count = tweet.user.favourites_count
        # reply_count = tweet.reply_count
        # quote_count = tweet.quote_count

        hashtag_list = [hashtag['text'] for hashtag in tweet.entities['hashtags']]
        mention_list = [mention['screen_name'] for mention in tweet.entities['user_mentions']]

        try: 
            text = tweet.retweeted_status.full_text 
        except AttributeError: 
            text = tweet.full_text 
  
        tweets = [id, created_at, username, location, followers, following, 
                     totaltweetsbyuser, retweetcount, like_count, 
                  impression_count, hashtag_list, mention_list, text] 
        df.loc[len(df)] = tweets 
        
    return df

In [ ]:
keyword = "Genuine Undead"
tweets_df = search_by_keyword(api, keyword)
tweets_df.head()

,id,created_at,username,location,followers,following,totaltweetsbyuser,retweetcount,like_count,impression_count,hashtag_list,mention_list,text
0,1620005322365747200,2023-01-30 10:25:41+00:00,FabriceFardeau,UK - FR,6716,3298,5359,30,0,5340,[],"[TaoKoltes, NFTFactoryParis]",From 1st to 12th feb Genuine Undead'll be on s...
1,1620005015833268224,2023-01-30 10:24:28+00:00,ombria3,,2,40,35,146,0,0,[],[kenjisang2],I reached 1k from 70 followers The reason this...
2,1620004621929549824,2023-01-30 10:22:54+00:00,TeamAmadeus4,amadeusguiver.com,1592,4903,14003,30,0,12518,[],"[TaoKoltes, NFTFactoryParis]",From 1st to 12th feb Genuine Undead'll be on s...
3,1620003694279278592,2023-01-30 10:19:13+00:00,lisichka7878,,0,0,31,303,0,0,[],"[KCMontero, GenuineUndead]",Genuine Undead Officer Montero reporting for d...
4,1620003506299232256,2023-01-30 10:18:28+00:00,1116_eth,,1940,2217,8538,30,0,15383,[],"[TaoKoltes, NFTFactoryParis]",From 1st to 12th feb Genuine Undead'll be on s...


### Get Opensea data 